In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib import rc
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch import nn, optim
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from statistics import mode
from pandas.plotting import register_matplotlib_converters
import copy
import os
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
import random
random.seed(2505)
import scipy.stats
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import Conv3D
from keras.layers import ConvLSTM2D
from keras.layers import ConvLSTM3D
from keras.layers import BatchNormalization

Mounted at /content/drive


In [ ]:
#PreProcessing

In [ ]:
import os
os.chdir(r"/content/drive/My Drive")

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
datasets = pd.read_csv('ETTh1.csv') #Change the datasets here
datasets

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,01/07/2016 00:00,5.827,2.009,1.599,0.462,4.203,1.340,30.531000
1,01/07/2016 01:00,5.693,2.076,1.492,0.426,4.142,1.371,27.787001
2,01/07/2016 02:00,5.157,1.741,1.279,0.355,3.777,1.218,27.787001
3,01/07/2016 03:00,5.090,1.942,1.279,0.391,3.807,1.279,25.044001
4,01/07/2016 04:00,5.358,1.942,1.492,0.462,3.868,1.279,21.948000
...,...,...,...,...,...,...,...,...
17415,26/06/2018 15:00,-1.674,3.550,-5.615,2.132,3.472,1.523,10.904000
17416,26/06/2018 16:00,-5.492,4.287,-9.132,2.274,3.533,1.675,11.044000
17417,26/06/2018 17:00,2.813,3.818,-0.817,2.097,3.716,1.523,10.271000
17418,26/06/2018 18:00,9.243,3.818,5.472,2.097,3.655,1.432,9.778000


In [ ]:
datasets['date'] = pd.to_datetime(datasets['date'],format='mixed').dt.date
datasets

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,2016-01-07,5.827,2.009,1.599,0.462,4.203,1.340,30.531000
1,2016-01-07,5.693,2.076,1.492,0.426,4.142,1.371,27.787001
2,2016-01-07,5.157,1.741,1.279,0.355,3.777,1.218,27.787001
3,2016-01-07,5.090,1.942,1.279,0.391,3.807,1.279,25.044001
4,2016-01-07,5.358,1.942,1.492,0.462,3.868,1.279,21.948000
...,...,...,...,...,...,...,...,...
17415,2018-06-26,-1.674,3.550,-5.615,2.132,3.472,1.523,10.904000
17416,2018-06-26,-5.492,4.287,-9.132,2.274,3.533,1.675,11.044000
17417,2018-06-26,2.813,3.818,-0.817,2.097,3.716,1.523,10.271000
17418,2018-06-26,9.243,3.818,5.472,2.097,3.655,1.432,9.778000


In [ ]:
datasets['date']=pd.to_datetime(datasets['date'])

In [ ]:
datasets.index = datasets.pop('date')

In [ ]:
datasets

,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
date,,,,,,,
2016-01-07,5.827,2.009,1.599,0.462,4.203,1.340,30.531000
2016-01-07,5.693,2.076,1.492,0.426,4.142,1.371,27.787001
2016-01-07,5.157,1.741,1.279,0.355,3.777,1.218,27.787001
2016-01-07,5.090,1.942,1.279,0.391,3.807,1.279,25.044001
2016-01-07,5.358,1.942,1.492,0.462,3.868,1.279,21.948000
...,...,...,...,...,...,...,...
2018-06-26,-1.674,3.550,-5.615,2.132,3.472,1.523,10.904000
2018-06-26,-5.492,4.287,-9.132,2.274,3.533,1.675,11.044000
2018-06-26,2.813,3.818,-0.817,2.097,3.716,1.523,10.271000


In [ ]:
test_data_size = 3484 #Split train & test For ETTh 80:20

datasets_train = datasets[:-test_data_size]
datasets_test = datasets[-test_data_size:]

In [ ]:
print(len(datasets_train))
print(len(datasets_test))

13936
3484


In [ ]:
#pearson-correlation-coefficient to define the best external factor feature, heighest value is better
import scipy.stats

corr_HUFL, _ = scipy.stats.pearsonr(datasets_train['OT'],datasets_train['HUFL'])
corr_HULL, _ = scipy.stats.pearsonr(datasets_train['OT'],datasets_train['HULL'])
corr_MUFL, _ = scipy.stats.pearsonr(datasets_train['OT'],datasets_train['MUFL'])
corr_MULL, _ = scipy.stats.pearsonr(datasets_train['OT'],datasets_train['MULL'])
corr_LUFL, _ = scipy.stats.pearsonr(datasets_train['OT'],datasets_train['LUFL'])
corr_LULL, _ = scipy.stats.pearsonr(datasets_train['OT'],datasets_train['LULL'])

print("OT-HUFL",corr_HUFL)
print("OT-HULL",corr_HULL)
print("OT-MUFL",corr_MUFL)
print("OT-MULL",corr_MULL)
print("OT-LUFL",corr_LUFL)
print("OT-LULL",corr_LULL)


OT-HUFL 0.07492511329809452
OT-HULL 0.30376157923031494
OT-MUFL 0.05093444857717344
OT-MULL 0.2828634030264733
OT-LUFL 0.1938454808178931
OT-LULL 0.10488055764770253


In [ ]:
if (corr_HUFL > corr_HULL) and (corr_HUFL > corr_MUFL) and (corr_HUFL > corr_MULL) and (corr_HUFL > corr_LUFL) and (corr_HUFL > corr_LULL) :
  ratio = datasets_train['OT']/((datasets_train['HUFL']))
  ratio =pd.DataFrame(ratio,columns=['Ratio'])
  ex_factor = datasets_train['HUFL']
  ex_factor =pd.DataFrame(ex_factor,columns=['HUFL'])
elif(corr_HULL > corr_HUFL) and (corr_HULL > corr_MUFL) and (corr_HULL > corr_MULL) and (corr_HULL > corr_LUFL) and (corr_HULL > corr_LULL):
  ratio = datasets_train['OT']/((datasets_train['HULL']))
  ratio =pd.DataFrame(ratio,columns=['Ratio'])
  ex_factor = datasets_train['HULL']
  ex_factor =pd.DataFrame(ex_factor,columns=['HULL'])
elif(corr_MUFL > corr_HULL) and (corr_MUFL > corr_HUFL) and (corr_MUFL > corr_MULL) and (corr_MUFL > corr_LUFL) and (corr_MUFL > corr_LULL):
  ratio = datasets_train['OT']/((datasets_train['MUFL']))
  ratio =pd.DataFrame(ratio,columns=['Ratio'])
  ex_factor = datasets_train['MUFL']
  ex_factor =pd.DataFrame(ex_factor,columns=['MUFL'])
elif(corr_MULL > corr_HULL) and (corr_MULL > corr_HUFL) and (corr_MULL > corr_MUFL) and (corr_MULL > corr_LUFL) and (corr_MULL > corr_LULL):
  ratio = datasets_train['OT']/((datasets_train['MULL']))
  ratio =pd.DataFrame(ratio,columns=['Ratio'])
  ex_factor = datasets_train['MULL']
  ex_factor =pd.DataFrame(ex_factor,columns=['MULL'])
elif(corr_LUFL > corr_HULL) and (corr_LUFL > corr_HUFL) and (corr_LUFL > corr_MUFL) and (corr_LUFL > corr_MULL) and (corr_LUFL > corr_LULL):
  ratio = datasets_train['OT']/((datasets_train['LUFL']))
  ratio =pd.DataFrame(ratio,columns=['Ratio'])
  ex_factor = datasets_train['LUFL']
  ex_factor =pd.DataFrame(ex_factor,columns=['LUFL'])
else:
  ratio = datasets_train['OT']/((datasets_train['LULL']))
  ratio =pd.DataFrame(ratio,columns=['Ratio'])
  ex_factor = datasets_train['LULL']
  ex_factor =pd.DataFrame(ex_factor,columns=['LULL'])

target = datasets_train['OT']
target =pd.DataFrame(target,columns=['OT'])

In [ ]:
target

,OT
date,
2016-01-07,30.531000
2016-01-07,27.787001
2016-01-07,27.787001
2016-01-07,25.044001
2016-01-07,21.948000
...,...
2018-01-02,4.713000
2018-01-02,4.291000
2018-01-02,4.291000


In [ ]:
ratio

,Ratio
date,
2016-01-07,15.197113
2016-01-07,13.384875
2016-01-07,15.960367
2016-01-07,12.895984
2016-01-07,11.301751
...,...
2018-01-02,1.851866
2018-01-02,1.392730
2018-01-02,1.562636


In [ ]:
ex_factor

,HULL
date,
2016-01-07,2.009
2016-01-07,2.076
2016-01-07,1.741
2016-01-07,1.942
2016-01-07,1.942
...,...
2018-01-02,2.545
2018-01-02,3.081
2018-01-02,2.746


In [ ]:
## Model Prediction Part

In [ ]:
ratio = ratio[np.isfinite(ratio).all(1)]

In [ ]:
#Spliting train & test after pearson correlation
#Target
train_data_target = target
test_data_target = target[-test_data_size:]

#ex_Factor
train_data_ex = ex_factor
test_data_ex = ex_factor[-test_data_size:]

#Ratio
train_data_ratio = ratio
test_data_ratio = ratio[-test_data_size:]



In [ ]:
print("Len Train Data Target",train_data_target.shape)
print("Len Train Data exFactor",train_data_ex.shape)
print("Len Train Data Ratio",train_data_ratio.shape)
print("---------------------------------------------")
print("Len Test Data Target",test_data_target.shape)
print("Len Test Data exFactor",test_data_ex.shape)
print("Len Test Data Ratio",test_data_ratio.shape)

Len Train Data Target (13936, 1)
Len Train Data exFactor (13936, 1)
Len Train Data Ratio (13554, 1)
---------------------------------------------
Len Test Data Target (3484, 1)
Len Test Data exFactor (3484, 1)
Len Test Data Ratio (3484, 1)


In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))
#Target
train_data_target = scaler.fit_transform(train_data_target)
test_data_target = scaler.fit_transform(test_data_target)
#ex_Factor
train_data_ex = scaler.fit_transform(train_data_ex)
test_data_ex = scaler.fit_transform(test_data_ex)
#Ratio
train_data_ratio = scaler.fit_transform(train_data_ratio)
test_data_ratio = scaler.fit_transform(test_data_ratio)



In [ ]:
def create_sequences(data, seq_length):
    xs = []
    ys = []

    for i in range(len(data)-seq_length-1):
        x = data[i:(i+seq_length)]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)

    return np.array(xs), np.array(ys)

In [ ]:
seq_length = 12
#Target
X_train_target, y_train_target = create_sequences(train_data_target, seq_length)
X_test_target, y_test_target = create_sequences(test_data_target, seq_length)
X_train_target = torch.from_numpy(X_train_target).float()
y_train_target = torch.from_numpy(y_train_target).float()
X_test_target = torch.from_numpy(X_test_target).float()
y_test_target = torch.from_numpy(y_test_target).float()
#ex_Factor
X_train_ex, y_train_ex = create_sequences(train_data_ex, seq_length)
X_test_ex, y_test_ex = create_sequences(test_data_ex, seq_length)
X_train_ex = torch.from_numpy(X_train_ex).float()
y_train_ex = torch.from_numpy(y_train_ex).float()
X_test_ex = torch.from_numpy(X_test_ex).float()
y_test_ex = torch.from_numpy(y_test_ex).float()
#Ratio
X_train_ratio, y_train_ratio = create_sequences(train_data_ratio, seq_length)
X_test_ratio, y_test_ratio = create_sequences(test_data_ratio, seq_length)
X_train_ratio = torch.from_numpy(X_train_ratio).float()
y_train_ratio = torch.from_numpy(y_train_ratio).float()
X_test_ratio = torch.from_numpy(X_test_ratio).float()
y_test_ratio = torch.from_numpy(y_test_ratio).float()

In [ ]:
print("X train target",X_train_target.shape)
print("y train target",y_train_target.shape)
print("-------------------------------------")
print("X train exFactor",X_train_ex.shape)
print("y train exFactor",y_train_ex.shape)
print("-------------------------------------")
print("X train Ratio",X_train_ratio.shape)
print("y train Ratio",y_train_ratio.shape)


X train target torch.Size([13923, 12, 1])
y train target torch.Size([13923, 1])
-------------------------------------
X train exFactor torch.Size([13923, 12, 1])
y train exFactor torch.Size([13923, 1])
-------------------------------------
X train Ratio torch.Size([13541, 12, 1])
y train Ratio torch.Size([13541, 1])


In [ ]:
print("X test target",X_test_target.shape)
print("y test target",y_test_target.shape)
print("-------------------------------------")
print("X test exFactor",X_test_ex.shape)
print("y test exFactor",y_test_ex.shape)
print("-------------------------------------")
print("X test Ratio",X_test_ratio.shape)
print("y test Ratio",y_test_ratio.shape)


X test target torch.Size([3471, 12, 1])
y test target torch.Size([3471, 1])
-------------------------------------
X test exFactor torch.Size([3471, 12, 1])
y test exFactor torch.Size([3471, 1])
-------------------------------------
X test Ratio torch.Size([3471, 12, 1])
y test Ratio torch.Size([3471, 1])


In [ ]:
#Target
X_target = np.array(X_train_target)
Y_target = np.array(y_train_target)
#ex_Factor
X_factor = np.array(X_train_ex)
Y_factor = np.array(y_train_ex)
#Ratio
X_ratio = np.array(X_train_ratio)
Y_ratio = np.array(y_train_ratio)

In [ ]:
from statistics import mode
import inspect
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error #as mae
from sklearn.metrics import r2_score

class Arwan:
  def __init__(self, epochs, batch_size, learning_rate,beta_1,beta_2):
    self.epochs = epochs
    self.batch_size = batch_size
    self.learning_rate = learning_rate
    self.beta_1 = beta_1
    self.beta_2 = beta_2

  def tensor(self):
    self.test_X_target_cnn = np.array(X_test_target)
    self.test_Y_target_cnn = np.array(y_test_target)
    self.test_Y_target = np.array(y_test_target)
    self.test_X_ex = np.array(X_test_ex)
    self.test_Y_ex = np.array(y_test_ex)
    self.test_X_ratio = np.array(X_test_ratio)
    self.test_Y_ratio = np.array(y_test_ratio)

  def target_model(self):
    #Let's build the ConV LSTM for Target Variable
    model_TargetConvLSTM = tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(filters=32, kernel_size=3,
                               strides=1, padding="causal",
                               activation="relu",
                               input_shape=(X_train_target.shape[1],X_train_target.shape[2])),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16,return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(8,return_sequences=False)),
        tf.keras.layers.Dense(1),
        tf.keras.layers.Lambda(lambda x: x * 100)])
    optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate, beta_1=self.beta_1, beta_2=self.beta_2,amsgrad=False)
    model_TargetConvLSTM.compile(loss='mean_squared_error', optimizer = optimizer)
    self.historyTargetConvLSTM = model_TargetConvLSTM.fit(X_target,Y_target,shuffle = False,epochs=self.epochs,batch_size=self.batch_size,validation_split=0.2,verbose=1)
    self.y_pred_target_cnn = model_TargetConvLSTM.predict(self.test_X_target_cnn)[:96] #This is prediction length, you change the prediction length here

  def eFactor_model(self):
    #Let's build Stacked GRU for ExFactor Variable
    model_ex = keras.Sequential()
    #Add a GRU layer with 3 units.
    model_ex.add(layers.GRU(3,
                     activation = "tanh",
                     recurrent_activation = "sigmoid",
                     return_sequences=True,
                     input_shape=(X_train_ex.shape[1], X_train_ex.shape[2])))
    model_ex.add(layers.GRU(5,
                     activation = "tanh",
                     recurrent_activation = "sigmoid",
                     return_sequences=True))
    model_ex.add(layers.GRU(8,
                     activation = "tanh",
                     recurrent_activation = "sigmoid",
                     return_sequences=True))
    model_ex.add(layers.GRU(5,
                     activation = "tanh",
                     recurrent_activation = "sigmoid",
                     return_sequences=True))
    model_ex.add(layers.GRU(3,
                     activation = "tanh",
                     recurrent_activation = "sigmoid",
                     return_sequences=True))
    model_ex.add(layers.GRU(1,
                     activation = "tanh",
                     recurrent_activation = "sigmoid",
                     return_sequences=False))
    #Add a dropout layer (penalizing more complex models) -- prevents overfitting
    model_ex.add(layers.Dropout(rate=0.01))
    #Add a Dense layer with 1 units (Since we are doing a regression task)
    model_ex.add(layers.Dense(1))
    optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate, beta_1=self.beta_1, beta_2=self.beta_2,amsgrad=False)
    model_ex.compile(loss='mean_squared_error', optimizer = optimizer)
    self.historyEfactor = model_ex.fit(X_factor,Y_factor,shuffle= False,epochs=self.epochs, batch_size=self.batch_size, validation_split=0.2, verbose=1)
    self.y_pred_ex = model_ex.predict(self.test_X_ex)[:96] #

  def ratio_model(self):
    #Let's build the BiGRU for Ratio
    model_ratio = keras.Sequential()
    #Add a GRU layer with 3 units.
    model_ratio.add(Bidirectional(layers.GRU(128,
                     activation = "tanh",
                     recurrent_activation = "sigmoid",
                     input_shape=(X_train_ratio.shape[1], X_train_ratio.shape[2]))))
    model_ratio.add(layers.Dropout(rate=0.01))
    #Add a Dense layer with 1 units (Since we are doing a regression task.
    model_ratio.add(layers.Dense(1))
    #Evaluating loss function of MSE using the adam optimizer.
    optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate, beta_1=self.beta_1, beta_2=self.beta_2,amsgrad=False)
    model_ratio.compile(loss='mean_squared_error', optimizer = optimizer)
    self.historyRatio = model_ratio.fit(X_ratio,Y_ratio, shuffle = False,epochs=self.epochs,batch_size=self.batch_size,validation_split=0.2,verbose=1)
    self.y_pred_ratio = model_ratio.predict(self.test_X_ratio)[:96] #
    return

  def train(self):
    target = self.target_model.fit(X_target,Y_target,shuffle = False,epochs=40,batch_size=8,validation_split=0.2,verbose=1)

  def predict(self):
    self.y_pred_target_cnn = self.y_pred_target_cnn.flatten()[:96] #Must same with prediction length
    self.y_pred_ex = self.y_pred_ex.flatten()[:96] #Must same with prediction length
    self.y_pred_ratio = self.y_pred_ratio.flatten()[:96] #Must same with prediction length
    self.y_test_target = self.test_Y_target_cnn.flatten()[:96] #Must same with prediction length
    self.y_test_target = self.y_test_target.flatten()[:96] #Must same with prediction length

    self.mean_cnn_pred = self.y_pred_target_cnn.mean() # Mean Convolutional LSTM
    self.median_cnn_pred = np.median(self.y_pred_target_cnn)
    self.mode_cnn_pred = mode(self.y_pred_target_cnn)

    self.mean_cnn_pred = (self.mean_cnn_pred+self.mode_cnn_pred)/2

    if self.mode_cnn_pred > self.mean_cnn_pred:
        self.predictor = self.mode_cnn_pred - self.mean_cnn_pred
    else:
        self.predictor = self.mean_cnn_pred - self.mode_cnn_pred

    self.mean_y_pred_ex = self.y_pred_ex.mean()
    self.mean_y_pred_ratio = self.y_pred_ratio.mean()

    self.AQI_Pred_ratio_cross_mean_ex = self.y_pred_ratio * self.mean_y_pred_ex #Based ratio X exMean
    self.AQI_Pred_ex_cross_mean_ratio = self.y_pred_ex * self.mean_y_pred_ratio

    self.AQI_Pred_ratio_cross_mean_ex_plus_mean_cnn_pred = self.AQI_Pred_ratio_cross_mean_ex + self.mean_cnn_pred #Based ratio X exMean + MeanConvLSTM
    self.AQI_Pred_ex_cross_mean_ratio_plus_mean_cnn_pred = self.AQI_Pred_ex_cross_mean_ratio + self.mean_cnn_pred #
    self.AQI_Pred_ex_cross_mean_ratio_plus_mean_cnn_pred_plus_pred = self.AQI_Pred_ratio_cross_mean_ex + self.mean_cnn_pred + self.predictor #Based ratio X exMean + MeanConvLSTM + Predictor

  def dataframe(self):
    self.result = pd.DataFrame(data={'Actual AQI':self.y_test_target,'AQI(ConvLSTM)':self.y_pred_target_cnn,'AQI(R * Ex-Mean)':self.AQI_Pred_ratio_cross_mean_ex,'AQI(R * Ex-Mean + MeanCNN)':self.AQI_Pred_ratio_cross_mean_ex_plus_mean_cnn_pred ,'AQI(R * Ex-Mean + MeanCNN + Predictor)':self.AQI_Pred_ex_cross_mean_ratio_plus_mean_cnn_pred_plus_pred})
    return self.result

  def smape(self,a, f):
    return 1/len(self.a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)

  def matrik_error(self):
    self.mse_ratio_cross_exMean = mean_squared_error(self.result['Actual AQI'], self.result['AQI(R * Ex-Mean)'])
    self.mae_ratio_cross_exMean = mean_absolute_error(self.result['Actual AQI'], self.result['AQI(R * Ex-Mean)'])
    self.r2_ratio_cross_exMean = r2_score(self.result['Actual AQI'], self.result['AQI(R * Ex-Mean)'],multioutput='variance_weighted')
    self.mse_ratio_cross_exMean_plus_meanCNN = mean_squared_error(self.result['Actual AQI'], self.result['AQI(R * Ex-Mean + MeanCNN)'])
    self.mae_ratio_cross_exMean_plus_meanCNN = mean_absolute_error(self.result['Actual AQI'], self.result['AQI(R * Ex-Mean + MeanCNN)'])
    self.r2_ratio_cross_exMean_plus_meanCNN = r2_score(self.result['Actual AQI'], self.result['AQI(R * Ex-Mean + MeanCNN)'])
    self.r2_ratio_cross_exMean_plus_meanCNN_plus_Predictor = r2_score(self.result['Actual AQI'], self.result['AQI(R * Ex-Mean + MeanCNN + Predictor)'],multioutput='variance_weighted')
    self.mse_ratio_cross_exMean_plus_meanCNN_plus_Predictor = mean_squared_error(self.result['Actual AQI'], self.result['AQI(R * Ex-Mean + MeanCNN + Predictor)'])
    self.mae_ratio_cross_exMean_plus_meanCNN_plus_Predictor = mean_absolute_error(self.result['Actual AQI'], self.result['AQI(R * Ex-Mean + MeanCNN + Predictor)'])

  def evaluation(self):
    print("MSE Based ratio X exMean + MeanConvLSTM",self.mse_ratio_cross_exMean_plus_meanCNN)
    print("MAE Based ratio X exMean + MeanConvLSTM",self.mae_ratio_cross_exMean_plus_meanCNN)
    print("R2 Based ratio X exMean + MeanConvLSTM",self.r2_ratio_cross_exMean_plus_meanCNN)
    print("----------------------------------------------")

  def train_target_plot(self):
    plt.plot(self.historyTargetConvLSTM.history['loss'], label = 'training loss')
    plt.plot(self.historyTargetConvLSTM.history['val_loss'], label = 'validation loss')
    plt.legend()
    return

  def train_eXfactor_plot(self):
    plt.plot(self.historyEfactor.history['loss'], label = 'training loss')
    plt.plot(self.historyEfactor.history['val_loss'], label = 'validation loss')
    plt.legend()
    return

  def train_ratio_plot(self):
    plt.plot(self.historyRatio.history['loss'], label = 'training loss')
    plt.plot(self.historyRatio.history['val_loss'], label = 'validation loss')
    plt.legend()
    return

  def compared(self):
    # Plot the predictions vs ground truths
    plt.figure(figsize=(12, 6))
    plt.plot(self.AQI_Pred_ratio_cross_mean_ex_plus_mean_cnn_pred, label='Predictions')
    plt.xlabel('Time')
    plt.ylabel('ETTm')
    plt.legend()
    plt.title('Ground Truth vs Predictions')
    plt.show()

  def true_vs_pred(self):
    result_arwan = pd.DataFrame(data={'Actual Values':self.y_test_target,'Predicted Values':self.AQI_Pred_ratio_cross_mean_ex_plus_mean_cnn_pred})
    result_arwan

  def run_all(self):
    self.tensor()
    self.target_model()
    self.eFactor_model()
    self.ratio_model()
    self.predict()
    self.dataframe()
    self.matrik_error()
    self.evaluation()
    self.true_vs_pred()

  @staticmethod
  def run_all_methods(arg):
    V = Arwan()
    attrs = (getattr(V, name) for name in dir(V))
    methods = filter(inspect.ismethod, attrs)
    result = list()
    for method in methods:
      print(method)
      try:
        df = method(arg)
        result.append(df)
      except Exception as e:
        print(e)
    return pd.concat(result)


In [ ]:
#(epochs, batch_size, learning_rate,beta_1,beta_2)
arwan = Arwan(50,32,0.001,0.9,0.99)
arwan.tensor()
arwan.target_model()
arwan.eFactor_model()
arwan.ratio_model()
arwan.predict()
arwan.dataframe()
arwan.matrik_error()
arwan.evaluation()
arwan.true_vs_pred()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 35s 55ms/step - loss: 0.8461 - val_loss: 0.0117
Epoch 2/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 19s 54ms/step - loss: 0.0263 - val_loss: 0.0973
Epoch 3/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 18s 51ms/step - loss: 0.0332 - val_loss: 0.0268
Epoch 4/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 22s 55ms/step - loss: 0.0203 - val_loss: 0.0485
Epoch 5/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 19s 53ms/step - loss: 0.0215 - val_loss: 0.0317
Epoch 6/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 20s 52ms/step - loss: 0.0155 - val_loss: 0.0244
Epoch 7/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 21s 52ms/step - loss: 0.0123 - val_loss: 0.0075
Epoch 8/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 22s 56ms/step - loss: 0.0101 - val_loss: 0.0046
Epoch 9/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 22s 61ms/step - loss: 0.0098 - val_loss: 0.0101
Epoch 10/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 39s 54ms/step - loss: 0.0082 - val_loss: 0.0111
Epoch 11/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 22s 62ms/step - loss: 0.0076 - val_loss: 0.0144
Epoch 12/50
349/349 ━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


349/349 ━━━━━━━━━━━━━━━━━━━━ 30s 47ms/step - loss: 0.0358 - val_loss: 0.0314
Epoch 2/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 22s 50ms/step - loss: 0.0198 - val_loss: 0.0195
Epoch 3/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 18s 43ms/step - loss: 0.0125 - val_loss: 0.0159
Epoch 4/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 15s 43ms/step - loss: 0.0112 - val_loss: 0.0153
Epoch 5/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 20s 41ms/step - loss: 0.0110 - val_loss: 0.0150
Epoch 6/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 21s 43ms/step - loss: 0.0110 - val_loss: 0.0149
Epoch 7/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 15s 43ms/step - loss: 0.0112 - val_loss: 0.0149
Epoch 8/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 20s 43ms/step - loss: 0.0112 - val_loss: 0.0147
Epoch 9/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 15s 43ms/step - loss: 0.0114 - val_loss: 0.0146
Epoch 10/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 20s 43ms/step - loss: 0.0115 - val_loss: 0.0145
Epoch 11/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 15s 44ms/step - loss: 0.0114 - val_loss: 0.0146
Epoch 12/50
349/349 ━━━━━━━━━━━━━━━━━━━━ 